# Семинар: Нейросетевое ранжирование
### Семинарист: Матвеев Артем, Yandex

В этом семинаре мы познакомимся с методами кодирование входных признаков для моделей нейросетевого ранжирвания: Mutisize Encoding with Unified Embeddings для категориальных признаков и Picewise Linear Encoding для вещественных. А также имплементируем DCNv2 и проверим всю схему на наборе данных Yambda. Попробуем обогнать такой сильный бейзлайн на табличных данных как Catboost.

In [1]:
# !wget -O data/pool.parquet https://huggingface.co/datasets/matfu21/yambda-50m-features/blob/main/pool.parquet
# !pip install -r requirements.txt

## 1. Yambda - 50m

Набор данных Yambda-5B — это большой открытый датасет, содержащий 4,79 млрд взаимодействий пользователь–трек, собранных от 1 млн пользователей и охватывающих 9,39 млн треков. Набор включает как неявный отклик (например, факты прослушивания), так и явный отклик в виде лайков и дизлайков. Кроме того, в нём есть отдельные метки для органических и рекомендательных взаимодействий, а также предвычисленные аудио-эмбеддинги, что упрощает разработку контент-ориентированных рекомендательных систем. Данные собрны с Яндекс Музыки. 

Ссылки: 
- hugging-face: https://huggingface.co/datasets/yandex/yambda
- hugging-face: https://huggingface.co/datasets/matfu21/yambda-50m-features
- arxiv: https://arxiv.org/pdf/2505.22238
- argus arxiv: https://www.arxiv.org/pdf/2507.15994

#### Метка класса:

`target_full_play`
- 1 — если событие listen и трек был прослушан ≥ 95%.
- 0 — если событие listen, но трек не дотянул до 95%.

#### Категориальные признаки

`uid` - ID пользователя.  
`item_id` - ID трека.

#### Вещественные признаки

`timestamp` - Время события в секундах (квантовано по 5 секунд, но мы храним в секундах).

1. Пользовательские (user_hist_*)

`user_hist_radio_skip_fraction_before`  
`user_hist_skip_fraction_before`  
`user_hist_radio_skip_before`  
`user_hist_like_before`  
`user_hist_track_finished_before`  
`user_hist_last_ts_before`  
`user_hist_radio_play_fraction_before`  
`user_hist_skip_frequency_before`  
`user_hist_skip_before`  

2. Посессионные признаки (session_hist_*)

`session_hist_duration_seconds_before`  
`session_hist_skip_fraction_before`  
`session_hist_played_time_seconds_before`  
`session_hist_skip_before`  
`session_hist_events_before`  
`session_hist_plays_before`  
`session_start_ts`  

3. item-ые (item_hist_*)

`item_hist_avg_played_ratio_before`  
`item_hist_track_finished_before`  

4. Кросс-признаки (user_item_hist_*)

`user_item_hist_track_finished_before`  
`user_item_hist_last_ts_before`  
`user_item_hist_avg_played_ratio_before`  
`user_item_hist_time_since_last_event_seconds`  
`user_item_hist_time_span_seconds_before`  
`user_item_hist_skip_before`  
`user_item_hist_plays_before`  
`user_item_hist_like_before`  

In [2]:
%matplotlib inline
import typing as tp
import polars as pl
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
from typing import Tuple

In [3]:
class YambdaDatasetUtils:
    NUM_COLS = [
        # timestamp
        "timestamp",

        # user history
        "user_hist_radio_skip_fraction_before",
        "user_hist_skip_fraction_before",
        "user_hist_radio_skip_before",
        "user_hist_like_before",
        "user_hist_track_finished_before",
        "user_hist_last_ts_before",
        "user_hist_radio_play_fraction_before",
        "user_hist_skip_frequency_before",
        "user_hist_skip_before",

        # session-level
        "session_hist_duration_seconds_before",
        "session_hist_skip_fraction_before",
        "session_hist_played_time_seconds_before",
        "session_hist_skip_before",
        "session_hist_events_before",
        "session_hist_plays_before",
        "session_start_ts",

        # item-level
        "item_hist_avg_played_ratio_before",
        "item_hist_track_finished_before",

        # user × item cross-features
        "user_item_hist_track_finished_before",
        "user_item_hist_last_ts_before",
        "user_item_hist_avg_played_ratio_before",
        "user_item_hist_time_since_last_event_seconds",
        "user_item_hist_time_span_seconds_before",
        "user_item_hist_skip_before",
        "user_item_hist_plays_before",
        "user_item_hist_like_before",
    ]
    CAT_COLS = [
        "uid",
        "item_id",
    ]
    LABEL_COl = "target_full_play"

    @classmethod
    def preprocess_dense_features(cls, lf: pl.LazyFrame) -> pl.LazyFrame:
        """
        Preprocess dense features:
        - Fill missing values with 0
        - Apply log transformation: log(x + 1)
        """
        expressions = []
        for col in cls.NUM_COLS:
            if col == "timestamp": # timestamp has uniform distribution, we don't need to apply log transformation
                continue
            expressions.append(
                pl.col(col).fill_null(0).add(1).log()
            )
        lf = lf.with_columns(expressions)
        return lf

    @classmethod
    def preprocess_categorical_features(cls, lf: pl.LazyFrame) -> pl.LazyFrame:
        """
        Preprocess categorical features:
        - Fill missing values with 0
        """
        expressions = []
        for col in cls.CAT_COLS:
            expressions.append(
                pl.col(col).fill_null(0)
            )
        lf = lf.with_columns(expressions)
        return lf

    @classmethod
    def split(cls, lf: pl.DataFrame, threshold: int) -> Tuple[pl.DataFrame, pl.DataFrame]:
        """
        Split data into train and test sets by threshold
        """
        df_train = lf.filter(pl.col("timestamp") <= threshold)
        df_test  = lf.filter(pl.col("timestamp")  > threshold)
        return df_train, df_test

    @classmethod
    def read_preprocess_and_split(cls, path: str, valid_period_in_sec: int) -> Tuple[pl.DataFrame, pl.DataFrame]:
        """
        path: path to parquet file
        valid_period_in_sec: valid period in seconds
        """
        lf = pl.scan_parquet(path)
        lf = cls.preprocess_categorical_features(lf)
        lf = cls.preprocess_dense_features(lf)
        df = lf.collect()

        max_timestamp = df.select(pl.col("timestamp").max()).item()
        threshold = max_timestamp - valid_period_in_sec
        df_train, df_test = cls.split(df, threshold)

        return df_train, df_test

In [4]:
DATASETS_PATH = './data/pool.parquet'
MONTH_PERIOD = 4 * 7 * 24 * 60 * 60  # 4 weeks, 7 days, 24 hours, 60 minutes, 60 seconds
df_train, df_test = YambdaDatasetUtils.read_preprocess_and_split(
    path=DATASETS_PATH,
    valid_period_in_sec=MONTH_PERIOD,
)
df_train.shape, df_test.shape

((19610232, 30), (2853323, 30))

In [5]:
df_train.head(5)

target_full_play,uid,item_id,timestamp,user_hist_radio_skip_fraction_before,user_hist_skip_fraction_before,user_hist_radio_skip_before,user_hist_like_before,user_hist_track_finished_before,user_hist_last_ts_before,user_hist_radio_play_fraction_before,user_hist_skip_frequency_before,user_hist_skip_before,session_hist_duration_seconds_before,session_hist_skip_fraction_before,session_hist_played_time_seconds_before,session_hist_skip_before,session_hist_events_before,session_hist_plays_before,session_start_ts,item_hist_avg_played_ratio_before,item_hist_track_finished_before,user_item_hist_track_finished_before,user_item_hist_last_ts_before,user_item_hist_avg_played_ratio_before,user_item_hist_time_since_last_event_seconds,user_item_hist_time_span_seconds_before,user_item_hist_skip_before,user_item_hist_plays_before,user_item_hist_like_before
bool,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
true,100,1441281,39420,0.0,0.0,0.0,0.0,0.693147,10.582054,0.693147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.582054,4.241981,2.70805,0.693147,0.0,4.615121,10.582054,22.176245,0.0,0.693147,0.0
true,100,8326270,39420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.663439,0.0,0.693147,0.693147,10.582054,4.546835,0.693147,0.0,0.0,3.091042,10.582054,22.180415,0.693147,0.693147,0.0
true,100,286361,39625,0.0,0.0,0.0,0.0,1.098612,10.582054,0.693147,0.0,0.0,5.327876,0.0,5.620401,0.0,1.098612,1.098612,10.582054,4.615121,1.098612,0.693147,0.0,3.931826,10.587241,22.179239,1.098612,1.386294,0.0
true,100,732449,40110,0.0,0.0,0.0,0.0,1.386294,10.587241,0.693147,0.0,0.0,6.53814,0.0,6.133398,0.0,1.386294,1.386294,10.582054,4.219508,1.098612,1.386294,0.0,4.251348,10.599406,22.1807,1.098612,1.791759,0.0
false,100,3397170,40360,0.0,0.0,0.0,0.0,1.609438,10.599406,0.693147,0.0,0.0,6.846943,0.0,6.552508,0.0,1.609438,1.609438,10.582054,4.433789,1.386294,0.0,0.0,0.0,10.605619,22.175255,0.693147,0.693147,0.0


Проверяем, что пропуски в данных отсуствуют.

In [6]:
assert df_train.null_count().pipe(sum).item() == 0
assert df_test.null_count().pipe(sum).item() == 0

Смотрим на количество уникальных значений категориальных признаков.

In [7]:
unique_counts = {col: df_train[col].n_unique() for col in YambdaDatasetUtils.CAT_COLS}
sorted_unique_counts = dict(
    sorted(unique_counts.items(), key=lambda item: item[1], reverse=True)
)
sorted_unique_counts

{'item_id': 452784, 'uid': 8410}

Посчитаем необходимое количество GPU памяти.

In [8]:
uniq_ids = sum(sorted_unique_counts.values())
embedding_dim = 256
bytes_in_float = 4
mult = 4  # params + grads + moment1 + moment2
bytes_in_gb = 1024 * 1024 * 1024
print(f'{uniq_ids * embedding_dim * bytes_in_float * mult / bytes_in_gb} GB')

1.7593154907226562 GB


Это всего лишь Yambda-50m, реальные размеры это Yambda-1e12 =). Даже если возьмем Yambda-5B, это будет 1,000,000 уникальный uid и 9,390,623 уникальный item_id.

In [9]:
uniq_ids = 1_000_000 + 9_390_623
print(f'{uniq_ids * embedding_dim * bytes_in_float * mult / bytes_in_gb} GB')

39.637081146240234 GB


Нужен более масштабируемый подход.

## 2. Multisize Unified Embeddings

Для кодирования категориальных признаков будем использовать Multisize Unified кодирование от Google DeepMind: [Unified Embedding: Battle-Tested Feature Representations for Web-Scale ML Systems](https://arxiv.org/abs/2305.12102).

### Общая задача

Дан $D = \{(x_1, y_1), (x_2, y_2), \ldots, (x_{|D|}, y_{|D|})\}$ с примерами из $T$ категориальных признаков с словарями $\{V_1, V_2, \ldots, V_T\}$. Каждый пример $x = [v_1, v_2, \ldots, v_T]$, где $v_i \in V_i$.

- Матрица эмбеддингов $\mathbf{E} \in \mathbb{R}^{M \times d}$, отображения примера в эмбеддинг $g(\mathbf{x}; \mathbf{E})$. 
- Хеш-функция $h(v) : V \rightarrow [M]$ назначает значение признака индексу строки (используется в $g(\mathbf{x}; \mathbf{E})$).
- Функция модели $f(\mathbf{e}; \boldsymbol{\theta})$ преобразует вложения в предсказание.

Задача обучения:
$$\arg \min_{\mathbf{E}, \boldsymbol{\theta}} \mathcal{L}_D(\mathbf{E}, \boldsymbol{\theta}), \quad \text{где} \quad \mathcal{L}_D(\mathbf{E}, \boldsymbol{\theta}) = \sum_{(\mathbf{x},y) \in D} \ell(f(g(\mathbf{x}; \mathbf{E}); \boldsymbol{\theta}), y).$$

Используем $h_t(v)$ для каждого признака $t \in [T]$. Обозначаем $\mathbf{e}_m$ для $m$-й строки $\mathbf{E}$, и $\mathbb{1}_{u,v}$ как индикатор коллизии хешей между $u$ и $v$.

### Как это работает

Далее будем предполагать, что $|T|$ = 2.

<div style="width:90%; margin: auto;">

![](https://i.ibb.co/GKMKcvm/unified-embeddings.png)

</div>


### Почему это работает (интуиция)

Рассмотрим частный случай (решаем бинарную классфикацию с помощью логистической регрессии):

$$y_i \in \{0, 1\}$$
$$ D_0 = \{(x_i, y_i) \in D : y_i = 0\} $$
$$ D_1 = \{(x_i, y_i) \in D : y_i = 1\} $$
$$ C_{u,v,0} = |\{([u, v], y) \in D : y = 0\}|$$
$$ \sigma_\theta(z) = \frac{1}{1 + \exp(-\langle z, \theta \rangle)} $$
$$ z = g(x; \mathbf{E}) = [e_{h_1(x_1)}, e_{h_2(x_2)}] $$
$$ \theta = [\theta_1, \theta_2],~\theta_t \in \mathbb{R}^M$$


Функция потерь бинарной кросс-энтропии:
$$ \mathcal{L}_D(\mathbf{E}, \theta) = - \sum_{(x,y)\in D_0} \log \left( \frac{1}{1 + \exp(-\langle \theta, g(x; \mathbf{E}) \rangle)} \right) - \sum_{(x,y)\in D_1} \log \left( \frac{1}{1 + \exp(\langle \theta, g(x; \mathbf{E}) \rangle)} \right) $$

Перепишем функция потерь (через частоты совместного появления):
$$ e_{u,v} = [e_{h_1(u)}, e_{h_2(v)}] $$

$$ \mathcal{L}_D(\mathbf{E}, \theta) = - \sum_{u\in V_1} \sum_{v\in V_2} C_{u,v,0} \log \left( \frac{1}{1 + \exp(-\theta^\top e_{u,v})} \right) + C_{u,v,1} \log \left( \frac{1}{1 + \exp(\theta^\top e_{u,v})} \right) $$

После объединения сигмоидных функций:
$$ \mathcal{L}_D(\mathbf{E}, \theta) = - \sum_{u\in V_1} \sum_{v\in V_2} C_{u,v,0} \log \exp(\theta^\top e_{u,v}) - (C_{u,v,0} + C_{u,v,1}) \log(1 + \exp(\theta^\top e_{u,v})) $$

Далее будем предполагать, что обучаем наш алгоритм с SGD. Посчитаем градиенты по эмбеддингам. Полный градиент для эмбеддинга с учетом внутри- и межпризнаковых взаимодействий:
$$ \nabla_{E_{h(u)}} \mathcal{L}_D(\mathbf{E}, \theta) = $$
$$ \theta_1 \sum_{v\in V_2} C_{u,v,0} - (C_{u,v,0} + C_{u,v,1})\sigma_\theta(e_{u,v}) \tag{1}$$
$$ + \theta_1 \sum_{w\in V_1, w\neq u} \mathbb{1}_{u,w} \sum_{v\in V_2} C_{w,v,0} - (C_{w,v,0} + C_{w,v,1})\sigma_\theta(e_{u,v}) \tag{2}$$
$$ + \theta_2 \sum_{v\in V_2} \mathbb{1}_{u,v} \sum_{w\in V_1} C_{w,v,0} - (C_{w,v,0} + C_{w,v,1})\sigma_\theta(e_{w,u}) \tag{3}$$

Анализируем:
- $(1)$ collisionless компонента.
- $(2)$ intra-feature компонента.
- $(3)$ inter-feature компонента.
- Компоненты $(2)$ и $(3)$ смещают реальный градиент.
- Intra-feature bias сонаправлен с collisionless компонентой, поэтому модель не может убрать это смещение.
- В случае SGD inter-feature bias может невелирован за счет $\theta_1$ ортогонально $\theta_2$, т.к. во время SGD, $e_{h(u)}$ представляет собой линейную комбинацию градиентов по шагам обучения, что означает, что $e_{h(u)}$ может быть разложено на компоненты в направлении $\theta_1$ и inter-feature компоненты в направлении $\theta_2$. Поскольку $\langle\theta_1, \theta_2\rangle = 0$, проекция $\theta_1^\top e_{h(u)}$ эффективно устраняет inter-feature  компонент.

<div style="width:70%; margin: auto;">

![](https://i.ibb.co/xt6nbrZ3/theory-unified.png)

</div>

Вывод: 

Не все коллизии одинаково проблематичны. Межпризнаковые коллизии могут быть смягчены однослойной нейронной сетью, поскольку разные признаки обрабатываются разными параметрами модели. Коллизии в рамках одного признака убираем за счет нескольких взятий хешей.

In [10]:
class MultihashTransform:
    """
    Applys transformation to training sample
    """
    def __init__(self, cardinality, seeds=None, name='sparse'):
        assert seeds is not None
        self._cardinality = cardinality
        self._name = name
        self._seeds = torch.tensor(seeds)

    def __call__(self, sample: dict[str, tp.Any]) -> dict[str, tp.Any]:
        sample[self._name] = (
            (sample[self._name].unsqueeze(2) + self._seeds) % self._cardinality
        ).long().flatten(-2)
        return sample

In [11]:
batch_size = 16

seeds = [
    [2342 + 13 * i, 7777 + 17 * i]
    for i in range(len(YambdaDatasetUtils.CAT_COLS))
]
transform = MultihashTransform(10, seeds)
input = {
    "label": torch.ones((batch_size,)),
    "dense": torch.randn((batch_size, len(YambdaDatasetUtils.NUM_COLS))),
    "sparse": torch.arange(len(YambdaDatasetUtils.CAT_COLS)).unsqueeze(0).repeat(batch_size, 1)
}
print(input["sparse"].shape)
output = transform(input)
print(output["sparse"].shape)
assert output["sparse"].shape == (batch_size, 2 * len(YambdaDatasetUtils.CAT_COLS))

torch.Size([16, 2])
torch.Size([16, 4])


In [12]:
class UnifiedEmbeddings(nn.Module):
    def __init__(self, cardinality, embedding_dim):
        super().__init__()
        self._cardinality = cardinality
        self._embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(
            num_embeddings=cardinality, embedding_dim=embedding_dim
        )

    def forward(self, ids: torch.Tensor):
        # ids shape: [batch_size, num_features]
        return self.embeddings(ids)

## 3. Piecewise Linear Encoding

Для кодирования вещественных признаков будем использовать Piecewise Linear Encoding от Yandex Research: [On Embeddings for Numerical Features in Tabular Deep Learning](https://arxiv.org/abs/2203.05556).

GitHub: https://github.com/yandex-research/rtdl-num-embeddings.

<div style="width:90%; margin: auto;">

![](https://i.ibb.co/XZtk6fSN/picewise-linear.png)

</div>

Эмбеддинги числовых признаков формализуются как $z_i = f_i(x_i^{(num)}) \in \mathbb{R}^{d_i}$, где:
- $f_i(x)$ — функция эмбеддинга для i-го числового признака
- $z_i$ — результирующий вектор эмбеддинга
- $d_i$ — размерность эмбеддинга

Ключевые особенности:
- Эмбеддинги вычисляются независимо для каждого признака
- В MLP-архитектурах эмбеддинги конкатенируются в один вектор
- В Transformer-архитектурах эмбеддинги используются без дополнительных преобразований

**Кусочно-линейное кодирование (PLE)**

PLE разбивает диапазон значений числового признака на $T$ интервалов (бинов) $B_1, \ldots, B_T$ с границами $[b_0, b_1], [b_1, b_2], ..., [b_{T-1}, b_T]$.

Формальное определение: $\text{PLE}(x) = [e_1, \ldots, e_T] \in \mathbb{R}^T$

где компоненты $e_t$ вычисляются как:

$$
e_t = 
\begin{cases}
0, & \text{если } x < b_{t - 1} \text{ И } t > 1 \\
1, & \text{если } x \geq b_t \text{ И } t < T \\
\frac{x-b_{t-1}}{b_t-b_{t-1}}, & \text{иначе}
\end{cases}
$$

Важные свойства:

- При $T = 1$ PLE эквивалентно скалярному представлению
- В отличие от категориальных признаков, PLE учитывает упорядоченность числовых данных
- PLE можно рассматривать как предобработку признаков

Применение в моделях с вниманием:

В моделях с механизмом внимания требуется дополнительно учитывать информацию об индексах признаков:

1. Для каждого бина $B_t$ выделяется обучаемый эмбеддинг $v_t \in \mathbb{R}^d$
2. Итоговый эмбеддинг вычисляется как: $f_i(x) = v_0 + \sum_{t=1}^T e_t \cdot v_t = \text{Linear}(\text{PLE}(x))$

Построение бинов:

Наиболее распространенный подход к построению бинов — разбиение по квантилям эмпирического распределения числового признака. Формально:
- Для i-го признака: $b_t = q_t \left(\{x_j^{(num)}\}_{j \in J_{train}}\right)$, где $q$ — функция эмпирического квантиля


In [13]:
class PiecewiseLinearEncodingTransform:
    """
    Applys transformation to training sample
    """
    @staticmethod
    def compute_bins(
        X: torch.Tensor,
        n_bins: int,
    ) -> list[torch.Tensor]:
        bins = [
            q.unique()
            for q in torch.quantile(
                X, torch.linspace(0.0, 1.0, n_bins + 1).to(X), dim=0
            ).T
        ]
        return bins

    def __init__(self, dense_train_df, n_bins=32, train_df_slice: int = 1_000_000, name='dense'):
        self._name = name
        self._bins = PiecewiseLinearEncodingTransform.compute_bins(
            dense_train_df.to_torch()[:train_df_slice], n_bins
        )
        n_features = len(self._bins)
        self._n_bins = [len(x) - 1 for x in self._bins]

        for n_bin in self._n_bins:
            assert n_bin >= 1, "There is a column with only one unique value"

        single_bin_mask = torch.tensor(self._n_bins) == 1
        self.single_bin_mask = single_bin_mask if single_bin_mask.any() else None

        max_n_bins = max(self._n_bins)

        self.mask = (
            None if all(len(x) == len(self._bins[0]) for x in self._bins)
            else torch.row_stack(
                [
                    torch.cat(
                        [
                            torch.ones((len(x) - 1) - 1, dtype=torch.bool),
                            torch.zeros(max_n_bins - (len(x) - 1), dtype=torch.bool),
                            torch.ones(1, dtype=torch.bool),
                        ]
                    )
                    for x in self._bins
                ]
            ).flatten(-2)
        )

        self.weight = torch.zeros(n_features, max_n_bins)
        self.bias = torch.zeros(n_features, max_n_bins)

        for i, bin_edges in enumerate(self._bins):
            bin_width = bin_edges.diff()
            w = 1.0 / bin_width
            b = -bin_edges[:-1] / bin_width
            self.weight[i, -1] = w[-1]
            self.bias[i, -1] = b[-1]
            self.weight[i, :self._n_bins[i] - 1] = w[:-1]
            self.bias[i, :self._n_bins[i] - 1] = b[:-1]

    @property
    def n_bins(self):
        return self._n_bins

    def __call__(self, sample: dict[str, tp.Any]) -> dict[str, tp.Any]:
        x = sample[self._name].to(torch.float32)
        x = torch.addcmul(self.bias, self.weight, x[..., None])
        x = torch.cat(
            [
                x[..., :1].clamp_max(1.0),
                x[..., 1:-1].clamp(0.0, 1.0),
                (
                    x[..., -1:].clamp_min(0.0)
                    if self.single_bin_mask is None
                    else torch.where(
                        self.single_bin_mask[..., None],
                        x[..., -1:],
                        x[..., -1:].clamp_min(0.0),
                    )
                )
            ],
            dim=-1,
        )
        x = x.flatten(-2)
        sample[self._name] = x if self.mask is None else x[:, self.mask]

        return sample

Пример: 

$weight$ =
\begin{bmatrix}
\frac{1}{b_1 - b_0} & \frac{1}{b_2 - b_1} & \frac{1}{b_3 - b_2} & \frac{1}{b_4 - b_3} \\
\frac{1}{c_1 - c_0} & \frac{1}{c_2 - c_1} & \frac{1}{c_3 - c_2} & \frac{1}{c_4 - c_3}
\end{bmatrix}

$bias$ =

\begin{bmatrix}
-\frac{b_0}{b_1 - b_0} & -\frac{b_1}{b_2 - b_1} & -\frac{b_2}{b_3 - b_2} & -\frac{b_3}{b_4 - b_3} \\
-\frac{c_0}{c_1 - c_0} & -\frac{c_1}{c_2 - c_1} & -\frac{c_2}{c_3 - c_2} & -\frac{c_3}{c_4 - c_3}
\end{bmatrix}

$X$ =
\begin{bmatrix}
x \\
y
\end{bmatrix}

$bias + weight \odot X$ = 
\begin{bmatrix}
\frac{x - b_0}{b_1 - b_0} &
\frac{x - b_1}{b_2 - b_1} &
\frac{x - b_2}{b_3 - b_2} &
\frac{x - b_3}{b_4 - b_3}
\\[8pt]
\frac{y - c_0}{c_1 - c_0} &
\frac{y - c_1}{c_2 - c_1} &
\frac{y - c_2}{c_3 - c_2} &
\frac{y - c_3}{c_4 - c_3}
\end{bmatrix}

In [14]:
N_BINS = 32
batch_size = 16

transform = PiecewiseLinearEncodingTransform(
    df_train[YambdaDatasetUtils.NUM_COLS],
    n_bins=N_BINS,
    name='dense'
)
input = {
    "label": torch.ones((batch_size,)),
    "dense": torch.randn((batch_size, len(YambdaDatasetUtils.NUM_COLS))),
    "sparse": torch.arange(len(YambdaDatasetUtils.CAT_COLS)).unsqueeze(0).repeat(batch_size, 1)
}
print(input["dense"].shape)
output = transform(input)
print(output["dense"].shape)

torch.Size([16, 27])
torch.Size([16, 692])


In [15]:
class PiecewiseLinearEncoding(nn.Identity):
    pass

## 4. DCN v2 - deep cross network

Для агрегации категориальных и вещественных признаков в один скаляр будем использовать DCNv2 от Google DeepMind: [DCN V2: Improved Deep & Cross Network and Practical Lessons for Web-scale Learning to Rank Systems](https://arxiv.org/abs/2008.13535).

### Подход

<div style="width:50%; margin: auto;">

![](https://i.ibb.co/ZqfF5yf/dcn-v2.png)
![](https://i.ibb.co/SDYWNSMy/dcn-v2-equation.png)

</div>

Stacked вариант:
- Сначала последовательность кросс-слоев: $$x_{i+1} = x_0 \odot (W \times x_i + b) + x_i.$$

- Затем последовательность Deep слоев: $$h_{l+1} = f(W_lh_l + b_l).$$


### Почему работает и зачем
- Знаем, что cross признаки важны. 
- DNN выучивает только неявные взаимодейтсвия, плохо аппроксимирует dot-product => нужные глубокие сети. 
- CrossNet добавляет явные взаимодействия признаков => не нужны глубоки DNN => может применять в рантайме.
- Явное взаимодействие := $x_1 x_2 \dots x_d$.

<div style="width:50%; margin: auto;">

![](https://i.ibb.co/SDWc6y1L/dcn-theory-1.png)
![](https://i.ibb.co/qLTgPJYF/dcn-theory-2.png)

</div>

In [16]:
class CrossLayer(torch.nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, input_dim)

    def forward(self, x0, xl):
        return x0 * self.linear(xl) + xl


class CrossNetwork(torch.nn.Module):
    def __init__(self, input_dim, num_layers):
        super().__init__()
        self.layers = nn.ModuleList([CrossLayer(input_dim) for _ in range(num_layers)])

    def forward(self, x):
        xl = x
        for layer in self.layers:
            xl = layer(x, xl)
        return xl


class DeepNetwork(torch.nn.Module):
    def __init__(self, input_dim, hidden_units):
        super().__init__()
        layers = []
        for units in hidden_units:
            layers.append(nn.Linear(input_dim, units))
            layers.append(nn.ReLU())
            input_dim = units
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


class DCNV2(nn.Module):
    def __init__(self, embedding_size, cross_layers, deep_units, input_size, cardinality=65536):
        super().__init__()
        self.sparse_encode_layer = UnifiedEmbeddings(cardinality, embedding_size)
        self.dense_encode_layer = PiecewiseLinearEncoding()
        self.cross_network = CrossNetwork(input_size, cross_layers)
        self.deep_network = DeepNetwork(input_size, deep_units)
        self.output_layer = nn.Linear(deep_units[-1], 1)

    def forward(self, dense_input, sparse_input):
        sparse_embeddings = self.sparse_encode_layer(sparse_input).view(sparse_input.size(0), -1)
        dense_embeddings = self.dense_encode_layer(dense_input)
        combined_input = torch.cat([dense_embeddings, sparse_embeddings], dim=-1)
        cross_output = self.cross_network(combined_input)
        deep_output = self.deep_network(cross_output)
        return self.output_layer(deep_output).squeeze(dim=-1)

## 5. Обучаем нейросетевое ранжирование

In [17]:
class YambdaDataset(Dataset):
    def __init__(
            self,
            df: pl.DataFrame,
            transforms: list[tp.Callable[[tp.Any], tp.Any]] | None = None,
            batch_size: int = 4096
    ):
        self._batch_size = batch_size
        self._labels_raw = df[YambdaDatasetUtils.LABEL_COl].to_torch().to(torch.float32)
        self._dense_raw = df[YambdaDatasetUtils.NUM_COLS].to_torch()
        self._sparse_raw = df[YambdaDatasetUtils.CAT_COLS].to_torch()
        self._transforms = (
            transforms if transforms is not None else []
        )

        self._labels = []
        self._dense = []
        self._sparse = []

        # precompute all transforms in bathes
        for i in tqdm(range(self._labels_raw.size(0) // batch_size)):
            sample = {
                'label': self._labels_raw[i * batch_size: (i + 1) * batch_size],
                'dense_features': self._dense_raw[i * batch_size: (i + 1) * batch_size],
                'sparse_features': self._sparse_raw[i * batch_size: (i + 1) * batch_size]
            }

            for transform in self._transforms:
                sample = transform(sample)

            self._labels.append(sample['label'])
            self._dense.append(sample['dense_features'])
            self._sparse.append(sample['sparse_features'])

        self._labels = torch.cat(self._labels, dim=0)
        self._dense = torch.cat(self._dense, dim=0)
        self._sparse = torch.cat(self._sparse, dim=0)

    def __len__(self):
        return self._labels.size(0)

    def __getitem__(self, idx):
        sample = {
            'label': self._labels[idx],
            'dense_features': self._dense[idx],
            'sparse_features': self._sparse[idx]
        }
        return sample


In [18]:
batch_size = 4096
cardinality = 8 * 65536
seeds = [[2342 + 13 * i, 7777 + 17 * i, 131 + 833 * i] for i in range(len(YambdaDatasetUtils.CAT_COLS))]
num_hashes = 3
embedding_size = 64
n_bins = 39

In [19]:
dense_transform = PiecewiseLinearEncodingTransform(df_train[YambdaDatasetUtils.NUM_COLS], n_bins, name='dense_features')
sparse_transform = MultihashTransform(cardinality, seeds, name='sparse_features')
transforms = [dense_transform, sparse_transform]

train_dataset = YambdaDataset(df_train, transforms)
val_dataset = YambdaDataset(df_test, transforms)

100%|██████████| 696/696 [01:10<00:00,  9.89it/s]


In [20]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, prefetch_factor=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, prefetch_factor=4)

In [21]:
def train_model(model, train_loader, val_loader, epochs=5, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0

        for batch in tqdm(train_loader):
            int_features, cat_features, labels = batch['dense_features'].to(device), batch['sparse_features'].to(device), batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(int_features, cat_features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss, correct, total = 0, 0, 0
        all_scores, all_labels = [], []
        with torch.no_grad():
            for batch in tqdm(val_loader):
                int_features, cat_features, labels = batch['dense_features'].to(device), batch['sparse_features'].to(device), batch['label'].to(device)


                outputs = model(int_features, cat_features)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_scores.append(outputs.clone().cpu())
                all_labels.append(labels.clone().cpu())
            all_scores = torch.cat(all_scores, dim=-1)
            all_labels = torch.cat(all_labels, dim=-1)


        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader):.4f}, '
              f'Val Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100*correct/total:.2f}%, '
              f'Val ROC AUC: {roc_auc_score(all_labels, all_scores)}')

In [22]:
input_size = 829 + num_hashes * embedding_size * len(YambdaDatasetUtils.CAT_COLS)
print(input_size)
model = DCNV2(
    embedding_size=embedding_size,
    cross_layers=3,
    deep_units=[1024, 1024, 1024],
    input_size=input_size,
    cardinality=cardinality,
)

1213


In [23]:
train_model(model, train_loader, val_loader, epochs=1, lr=1e-4)

100%|██████████| 696/696 [00:28<00:00, 24.62it/s]


Epoch 1/1, Train Loss: 0.3401, Val Loss: 0.3480, Accuracy: 83.19%, Val ROC AUC: 0.9183831665741273


## 6. Сравниваем с катбустом на том же наборе признаков

In [24]:
X_train = df_train[YambdaDatasetUtils.NUM_COLS + YambdaDatasetUtils.CAT_COLS].to_pandas()
y_train = df_train[YambdaDatasetUtils.LABEL_COl].to_pandas()
X_test = df_test[YambdaDatasetUtils.NUM_COLS + YambdaDatasetUtils.CAT_COLS].to_pandas()
y_test = df_test[YambdaDatasetUtils.LABEL_COl].to_pandas()

In [25]:
import catboost as cb


train_pool = cb.Pool(X_train, y_train, cat_features=YambdaDatasetUtils.CAT_COLS)
test_pool = cb.Pool(X_test, y_test, cat_features=YambdaDatasetUtils.CAT_COLS)

model = cb.CatBoostClassifier(
    iterations=2000,
    loss_function="Logloss",
    eval_metric="AUC",
    learning_rate=0.1,
    depth=6,
    early_stopping_rounds=50,
    task_type="GPU",
    devices='7',
    random_seed=42,
    verbose=100,
)
model.fit(train_pool, eval_set=test_pool, use_best_model=True)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8755186	best: 0.8755186 (0)	total: 282ms	remaining: 9m 24s
100:	test: 0.9056464	best: 0.9056604 (99)	total: 28.1s	remaining: 8m 49s
200:	test: 0.9092687	best: 0.9092687 (200)	total: 57.4s	remaining: 8m 34s
300:	test: 0.9106317	best: 0.9108937 (289)	total: 1m 26s	remaining: 8m 10s
400:	test: 0.9115660	best: 0.9115660 (400)	total: 1m 56s	remaining: 7m 44s
500:	test: 0.9120677	best: 0.9120677 (500)	total: 2m 25s	remaining: 7m 15s
600:	test: 0.9126255	best: 0.9126255 (600)	total: 2m 55s	remaining: 6m 49s
700:	test: 0.9131910	best: 0.9132269 (691)	total: 3m 26s	remaining: 6m 21s
bestTest = 0.913253963
bestIteration = 708
Shrink model to first 709 iterations.


In [26]:
importances = model.get_feature_importance(prettified=True)
print(importances)

                                      Feature Id  Importances
0           session_hist_duration_seconds_before    17.152574
1                  user_item_hist_last_ts_before    12.656227
2           user_item_hist_track_finished_before    11.372442
3                                            uid    10.912499
4         user_item_hist_avg_played_ratio_before     9.722466
5              session_hist_skip_fraction_before     8.220881
6                                        item_id     7.664724
7        session_hist_played_time_seconds_before     4.469543
8           user_hist_radio_skip_fraction_before     3.317510
9              item_hist_avg_played_ratio_before     2.492329
10  user_item_hist_time_since_last_event_seconds     2.415917
11                      session_hist_skip_before     2.412685
12                    session_hist_events_before     1.695017
13                     session_hist_plays_before     0.897447
14       user_item_hist_time_span_seconds_before     0.843303
15      

У нейросети получить feature importance не так просто, но есть способы.

## 7. Улучшаем нейросетевое ранжирование

In [27]:
# !wget -O data/album_item_mapping.parquet https://huggingface.co/datasets/yandex/yambda/resolve/main/album_item_mapping.parquet
# !wget -O data/artist_item_mapping.parquet https://huggingface.co/datasets/yandex/yambda/resolve/main/artist_item_mapping.parquet
# !wget -O data/embeddings.parquet https://huggingface.co/datasets/yandex/yambda/blob/main/embeddings.parquet
# !wget -O data/multi_event.parquet https://huggingface.co/datasets/yandex/yambda/resolve/main/flat/50m/multi_event.parquet

- Добавить artist_id, album_id. Это множественные категориальные признаки (список из категориальных признаков). 
- Добавить контентный эмбеддинг трека. 
- И еще кучу всего.